In [1]:
import vectorbt as vbt
import numpy as np
import datetime as dt


def get_data(tickers, startDate, endDate):
    dados = vbt.YFData.download(tickers, start = startDate, end = endDate,
                                missing_index='drop',interval='1h').get('Close')
    close = dados[:]
    return close


def custom_indicator(close, rsi_window = 14, slow_ma=200, fast_ma=50, entry=30, exit=70):
    rsi = vbt.RSI.run(close, window = rsi_window).rsi
    media_rap = vbt.MA.run(close, window = slow_ma).ma.to_numpy()
    media_lenta = vbt.MA.run(close, window = fast_ma).ma.to_numpy()

    trend = np.where( (rsi > exit) & (media_lenta < media_rap), -1, 0)  ## This array creates entries and exits
    trend = np.where( (rsi < entry) & (media_rap > media_lenta), 1, trend) 
    
    # rsi.plot(title='RSI IBOV')  ## Plot the asset RSI, in this case "IBOV"
    return trend


### INDICATOR FACTORY ### Excelent for parameter optimization
ind = vbt.IndicatorFactory(    
        class_name = "Combination",
        short_name = "comb",
        input_names = ["close"],
        param_names = ["rsi_window", "slow_ma", "fast_ma", "entry", "exit"],
        output_names = ["value"]    
        ).from_apply_func(
                custom_indicator,  
                rsi_window = 14,
                slow_ma = 200,
                fast_ma = 50,
                entry = 30,
                exit = 70
                )


def create_signals(result):
    entries = result.value == 1.0
    exits = result.value == -1.0
    return entries, exits


def build_portfolio(close, entries, exits):
    pf = vbt.Portfolio.from_signals(close, entries=entries, exits=exits,
                                    direction='all', sl_stop=0.05)
    print(pf.stats(silence_warnings=True))
    return pf


def main():
    endDate = dt.datetime.now()
    startDate = endDate - dt.timedelta(days=730)
    tickers = ['^BVSP']
    rsi_windows = 14
    slow_ma = 200
    fast_ma = 50
    entry = 30
    exit = 65


    close = get_data(tickers, startDate, endDate)
    trend = custom_indicator(close, rsi_window=rsi_windows, slow_ma=slow_ma, fast_ma=fast_ma, entry=entry, exit=exit)    
    result = ind.run(close, rsi_window=rsi_windows, slow_ma=slow_ma, fast_ma=fast_ma, entry=entry, exit=exit)
    entries, exits = create_signals(result)


    pf = build_portfolio(close, entries, exits)
    fig = pf.plot()
    fig.show()
  

if __name__ == '__main__':
    main()







Start                         2021-02-17 16:00:00+00:00
End                           2023-02-17 13:40:30+00:00
Period                                             3279
Start Value                                       100.0
End Value                                    167.049425
Total Return [%]                              67.049425
Benchmark Return [%]                          -8.078103
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              13.743323
Max Drawdown Duration                             753.0
Total Trades                                         58
Total Closed Trades                                  57
Total Open Trades                                     1
Open Trade P&L                                 0.430966
Win Rate [%]                                  71.929825
Best Trade [%]                                 5.766298
Worst Trade [%]                               -5